In [72]:
import torch
from torchvision import models, transforms
import torch.utils.data as D
import numpy as np
import pandas as pd

from PIL import Image

from scripts.evaluate import eval_model
from scripts.listdir import list_files 

In [73]:
classes = 1108
path_data = '../input/rxrxai'
batch_size = 32
device = 'cuda'

In [74]:
class ImagesDS(D.Dataset):
    def __init__(self, df, img_dir=path_data, mode='train', site=1, channels=[1,2,3,4,5,6]):
        self.records = df.to_records(index=False)
        self.channels = channels
        self.site = site
        self.mode = mode
        self.img_dir = img_dir
        self.len = df.shape[0]
        
    @staticmethod
    def _load_img_as_tensor(file_name):
        with Image.open(file_name) as img:
            return transforms.ToTensor()(img)

    def _get_img_path(self, index, channel):
        experiment, well, plate = self.records[index].experiment, self.records[index].well, self.records[index].plate
        return '/'.join([self.img_dir,self.mode,experiment,f'Plate{plate}',f'{well}_s{self.site}_w{channel}.png'])
        
    def __getitem__(self, index):
        paths = [self._get_img_path(index, ch) for ch in self.channels]
        img = torch.cat([self._load_img_as_tensor(img_path) for img_path in paths])
        if self.mode == 'train':
            return img, int(self.records[index].sirna)
        else:
            return img, self.records[index].id_code

    def __len__(self):
        return self.len

In [75]:
df_test = pd.read_csv(path_data+'/test.csv')
ds_test = ImagesDS(df_test, mode='test')
tloader = D.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=0)

# Resnet50 - 6 channels

In [76]:
model = getattr(models, 'resnet50')(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, classes)

# let's make our model work with 6 channels
trained_kernel = model.conv1.weight
new_conv = torch.nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)
with torch.no_grad():
    new_conv.weight[:,:] = torch.stack([torch.mean(trained_kernel, 1)]*6, dim=1)
model.conv1 = new_conv

In [ ]:
for file_path, file_name in list_files('models', sub_str='resnet50_6channels'):
    print(f'Evaluating {file_name}\n')
    model.load_state_dict(torch.load(file_path))
    model.cuda()
    eval_model(model, tloader, file_name)

Evaluating Model_resnet50_6channels_30e-5_39.pth

